# Machine Learning: Fundamentals

### What is Machine Learning and how can Machine Learning be useful?

Discussion!

Machine Learning is the process of building an algorithm which can learn without explicit instructions, this is accomplished with a mathematical model - a model is based on data, and is usually designed to exploit our current understanding of data to find a generalizable "trend" that applies to all instances of similar data. 

What is an example of a problem that can be handled with machine learning?


In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
df = pd.read_csv('../data/fifa19.csv')
df.head()

In [ ]:
df_sub = df[['Composure','Overall']].sample(20, random_state = 42)
df_sub.dropna(inplace = True)
df_sub.plot(x = 'Composure', y='Overall', kind = 'scatter')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

X = np.array(df_sub['Composure'])
y = df_sub['Overall']
degrees = [1,2,12]

plt.figure(figsize=(14, 5))
for i in range(len(degrees)):
    ax = plt.subplot(1, len(degrees), i + 1)
    plt.setp(ax, xticks=(), yticks=())

    polynomial_features = PolynomialFeatures(degree=degrees[i],
                                             include_bias=False)
    linear_regression = LinearRegression()
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    pipeline.fit(X[:, np.newaxis], y)

    # Evaluate the models using crossvalidation
    scores = cross_val_score(pipeline, X[:, np.newaxis], y,
                             scoring="neg_mean_squared_error", cv=10)

    X_test = np.linspace(25, 90, 100)
    plt.plot(X_test, pipeline.predict(X_test[:, np.newaxis]), label="Model")
    plt.scatter(X, y, edgecolor='b', s=20, label="Samples")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xlim((25, 90))
    plt.ylim((60, 80))
    plt.legend(loc="best")
    plt.title("Degree {}\nMSE = {:.2e}(+/- {:.2e})".format(
        degrees[i], -scores.mean(), scores.std()))
plt.show()


## Model Validation

Model validation is the process of assessing the fidelity of the model by confirming that the outputs from a machine learning model are representative of the population. This is done by evaluating the model on data it has not "seen."

What are some ways we could incorrectly validate our model?

In [ ]:
df.columns
df['not_awful'] = (df.Overall > 62)*1

In [ ]:
X = df[['not_awful','Position','Strength','Composure',
        'LongPassing','GKReflexes']]
X.dropna(inplace = True)

In [ ]:
X_train_1 = X[X.Position != 'GK']
X_train_1.drop(['Position'],axis=1, inplace= True)
Y_train_1 = X_train_1.pop('not_awful')


X_test_1 = X[X.Position == 'GK']
X_test_1.drop(['Position'],axis=1, inplace= True)
Y_test_1 = X_test_1.pop('not_awful')

In [ ]:
from sklearn.svm import SVC
svm = SVC(gamma ='auto')
svm.fit(X_train_1,Y_train_1)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_train = svm.predict(X_train_1)
print(str(accuracy_score(Y_train_1, y_pred_train)))

y_pred_test = svm.predict(X_test_1)
print(str(accuracy_score(Y_test_1, y_pred_test)))


Let's try a more reasonable split

In [ ]:
from sklearn.model_selection import train_test_split

X = df[['not_awful','Position','Strength','Composure',
        'LongPassing','GKReflexes']]
X.dropna(inplace = True)

Y = X.pop('not_awful')
X.drop('Position', axis = 1, inplace = True)

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.25, random_state=42)

In [ ]:
svm_tt = SVC(gamma ='auto')
svm_tt.fit(X_train,y_train)

In [ ]:
y_pred_train = svm_tt.predict(X_train)
print(str(accuracy_score(y_train, y_pred_train)))

y_pred_test = svm_tt.predict(X_test)
print(str(accuracy_score(y_test, y_pred_test)))

In [ ]:
from sklearn.model_selection import cross_validate

svm = SVC(gamma ='auto')
cv_results = cross_validate(svm, X_train, y_train, cv = 3, n_jobs = -1)

In [ ]:
from sklearn.model_selection import GridSearchCV 

param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']} 

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, n_jobs = -1) 
grid.fit(X_train, y_train) 

In [ ]:
grid.cv_results_['mean_test_score']

In [ ]:
grid.best_score_

In [ ]:
!pip install hyperopt

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
def hyperopt_train_test(params):
    X_ = X_train[:]
    clf = SVC(**params)
    return cross_val_score(clf, X_,y_train).mean()

space4svm = {
    'C': hp.uniform('C', 7.5, 12.5),
    'kernel': hp.choice('kernel', ['rbf']),
    'gamma': hp.uniform('gamma', 0.00001, .005),
}

def f(params):
    acc = hyperopt_train_test(params)
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, space4svm, algo=tpe.suggest, max_evals=10, trials=trials)
print('best:')
print(best)
